In [9]:
# Add a column of ones to the first column of a dataframe
# and turn it into a matrix
def df_addOnes(dataFrame):
    vals = dataFrame.values
    add_ones_column = zip(np.ones(len(dataFrame)), vals)
    return add_ones_column
    feature_matrix = np.matrix([val for val in add_ones_column])
    
    return feature_matrix

In [5]:
# Making it easy to calculate the total penalty over the entire dataset
def penalty(df_features, df_output, paramater_value_list):
    
    # df_features is a dataframe of the features (no column of ones added)
    # df_output is a dataframe of the output column (target variable)
    # parameter_value_list is a list of w0, w1, ..., wn+1 where n is the number of features
    #  i.e., the number of columns in df_features.
    
    # Cost of being wrong calculated over the entire data set
    # Will take X and add a first column of 1s to it to enable the matrix multiplication
    # Therefore: X is an m x n matrix and theta is a n x 1 matrix
    
    #### Turn the function inputs into matrices ####
    # Get X and y into the right shapes for use in computeCost
    # Add a first column of ones to the feature matrix
    # Add a column of 1s to X 
    feature_matrix = df_addOnes(df_features)

    output_matrix = np.matrix(df_output.values)
    parameter_matrix = np.matrix(paramater_value_list).T
    
    # Difference between the predicted and the actual value
    error = (feature_matrix * parameter_matrix) - output_matrix
    
    # penaltyPerOutput is an m x 1 matrix where each element is the penalty for
    # the input and its associated output for a particular value of W
    
    # Use the squared error penalty function
    penaltyPerOutput = np.power(error, 2)
    
    # totalPenalty is the sum of the penalties of each row of the dataset
    totalPenalty = np.sum(penaltyPerOutput)
    
    # The penalty of getting it wrong is 1/2m of the totalPenalty (normalized penalty)
    # m is the number of rows in df_features
    totalPenaltyNorm = totalPenalty / (2 * len(df_features))
    
    return totalPenaltyNorm

In [3]:
# Implement Gradient Descent
def gradientDescent(X, y, W, alpha, iters):
    # NOTE: X is the original dataframe -- need to add a column of ones to it and make it a matrix
    feature_matrix = df_addOnes(X)
    # feature_matrix is a m x n matrix
    # y is a m x 1 matrix
    # W is a n x 1 matrix
    
    # Keep track of everything
    sumError = np.zeros(shape=(len(W),1))
    sumErrorNorm = np.zeros(shape=(len(W),1))
    temp = np.matrix(np.zeros(W.shape))
    cost = np.zeros(iters)
    
    for i in range(iters):
        # Calculate the non-normalized values for each W parameter
        error = (feature_matrix * W) - y
        # return feature_matrix.shape, W.shape, y.shape, error, feature_matrix[:,1]
        
        for j in range(len(W)):
            # Multiply the error vector by the appropriate column of the feature_matrix and sum it
            sumError[j] = np.sum(np.multiply(error, feature_matrix[:,j]))
            
            # Normalize the sumError using alpha and m
            sumErrorNorm[j] = np.divide(np.multiply(sumError[j], alpha), len(feature_matrix))
            
            temp[j,0] = W[j,0] - sumErrorNorm[j]
        
        W = temp
    
        # Use the original dataframe in this call
        # the penalty function likes to see W.T as input
        cost[i] = penalty(X,y,W.T)
            
    # Return the value of W after iters iterations of gradient descent
    # Return the penalty = cost for this W
    return W, cost

In [ ]:
# Penalty Functions for a Single Feature
